# Classification using Keras

The target values must be in a column of the dataframe, and must be a numerical datatype, so we must convert them before training.

IMPORTANT: Notice that we are importing `ImageDataGenerator` from `keras_preprocessing` instead of `keras.preprocessing`, because the people at Keras are a bunch of old reactionary dinosaurs.

See the discussion [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1), and [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1).

In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

Using TensorFlow backend.


## Load data and preparation

The csv files `db.csv` contains all the metadata we have extracted from wikiart: every entry corresponds to an artwork, and the `_id` column contains the filename of the image in the `data/images/` directory associated to that artwork.

In [3]:
nrows = 5000
nrows = None # to load all 
df = pd.read_csv("../data/db.csv",nrows=nrows,na_values="?")

Train/test split the database:

In [3]:
df_train, df_test = train_test_split(df, 
                                     test_size=0.1,
                                     shuffle=True)

In [4]:
df_train.head()

NameError: name 'df_train' is not defined

Let's initialize a dataimage generator: it is a nice interface towards many (pre)processing method in Keras, including some utilities for data augmentation.

We will use the amazing `flow_from_dataframe` function to serve the data we need.

If the files do not have an extension, run this in a shell:

    $ for f in *; do mv "$f" "$f.jpg"; done

In [9]:
df.describe()

,_id,artistname,genre,image,image_size_data,style,title,year
count,152014,152014,149142,152014,152014,148475,152011,118672
unique,152014,2854,363,152010,152010,921,115378,2693
top,5772816aedc2cb3880f45e8e,vincent van gogh,portrait,https://uploads3.wikiart.org/images/nathan-alt...,"[{'sizekb': 10, 'width': 210, 'height': 189, '...",impressionism,untitled,1910
freq,1,1927,21514,3,3,14639,4440,1303


In [17]:
set(df["style"])

{'abstract art',
 'abstract art,abstract expressionism',
 'abstract art,color field painting',
 'abstract art,constructivism',
 'abstract art,contemporary',
 'abstract art,cubism',
 'abstract art,dada',
 'abstract art,digital art',
 'abstract art,expressionism',
 'abstract art,futurism',
 'abstract art,naïve art (primitivism)',
 'abstract art,new european painting',
 'abstract art,op art',
 'abstract art,orphism',
 'abstract art,post-impressionism',
 'abstract art,precisionism',
 'abstract art,surrealism',
 'abstract expressionism',
 'abstract expressionism,abstract art',
 'abstract expressionism,action painting',
 'abstract expressionism,art brut',
 'abstract expressionism,art deco',
 'abstract expressionism,color field painting',
 'abstract expressionism,conceptual art',
 'abstract expressionism,contemporary realism',
 'abstract expressionism,cubism',
 'abstract expressionism,expressionism',
 'abstract expressionism,feminist art',
 'abstract expressionism,hard edge painting',
 'abstr

In [47]:
df["style"].map(lambda s: str(s).split(",")[0].strip()).describe()

count            152014
unique              168
top       impressionism
freq              14978
Name: style, dtype: object

### Classes

Decide here what feature we want to predict, and save in the `classes` set all the possible values: they are the values that appear at least once in the database.

In [5]:
feature = "style"
classes=set(df[feature])
nclass = len(classes)
print(len(classes))

80


In [6]:
classes_list= list(classes)
for i in range(10):
    print(classes_list[i])
    
#I can write a loop to access classes which are reptitive

nan
cubism,futurism
minimalism
impressionism,realism
naïve art (primitivism),post-impressionism
classicism
dada
tenebrism
rococo
environmental (land) art


The number of classes will be needed later, the NN must know what's the output dimension.

### Image Size

The images will be scaled down to this size

In [7]:
img_size = (32,32)

## Create data generators

In [8]:
datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)
#As of now, we are not doing cross-validation. Maybe we should do that.
#Also, we should use data-augmentation here

In [9]:
train_generator = datagen.flow_from_dataframe(
                                df_train,
                                directory="../data/images/",
                                x_col="_id",
                                has_ext=False,
                                target_size=img_size,
                                y_col=feature,
                                batch_size=32,
                                classes = classes_list)

Found 4500 images belonging to 80 classes.


# Model Architecture

## CNN with no transfer learning

We'll start with the most basic architecture imaginable. 

It would be nice to use transfer learning.

In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(*img_size,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nclass, activation='softmax'))


## Transfer learning

In [12]:
model = ResNet50(weights='imagenet')

NameError: name 'ResNet50' is not defined

# Pick an optimizer and compile the model

In [11]:
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),
              loss="categorical_crossentropy",metrics=["accuracy"])

In [25]:
#training
model.fit_generator(generator=train_generator,
                    epochs=2)

Epoch 1/2
141/141 [==============================] - 12s 85ms/step - loss: 3.2359 - acc: 0.2312
Epoch 2/2
141/141 [==============================] - 12s 88ms/step - loss: 2.7976 - acc: 0.3181


In [21]:
test_generator = datagen.flow_from_dataframe(df_test,
                                directory="../data/images/",
                                x_col="_id",has_ext=False, target_size=img_size,
                                y_col=feature,batch_size=32, classes = classes)

Found 500 images belonging to 80 classes.


# Evaluation

This returns the loss and accuracy

In [18]:
model.evaluate_generator(generator=test_generator)

[3.1134791278839113, 0.2899999997615814]